In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# activate R magic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [3]:
%%R
install.packages("MicroMacroMultilevel")
install.packages("feather")
install.packages("dplyr")
library("MicroMacroMultilevel")
library(feather)
library("dplyr")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/MicroMacroMultilevel_0.4.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 13012 bytes (12 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console

In [4]:
%%R
x <- read_feather("/content/drive/My Drive/Dissertation/airbnb/cultural distance/revisions/revision2/lcm/xdata_full.feather")
z <- read_feather("/content/drive/My Drive/Dissertation/airbnb/cultural distance/revisions/revision2/lcm/zdata_full.feather")
x.gid <- x[['gid']]
x.data <- select(x, cult_dst_6, interaction)
#x.data$nothing <- sample(100, size = nrow(x.data), replace = TRUE)
z.gid <- z[['gid']]

data <- z
epsilon <- 0.0000001
percent_perf <- (30-data$availability_30)/30
z$perf <- ifelse(percent_perf == 0, percent_perf+epsilon, percent_perf)
z$perf <- ifelse(percent_perf == 1, percent_perf-epsilon, z$perf)
z$logit_perf <- log(z$perf/(1-z$perf))

#add seasonality
z$jan <- ifelse(substr(z$yyyymm, 5, 6)=="01", 1, 0)
z$feb <- ifelse(substr(z$yyyymm, 5, 6)=="02", 1, 0)
z$mar <- ifelse(substr(z$yyyymm, 5, 6)=="03", 1, 0)
z$apr <- ifelse(substr(z$yyyymm, 5, 6)=="04", 1, 0)
z$may <- ifelse(substr(z$yyyymm, 5, 6)=="05", 1, 0)
z$jun <- ifelse(substr(z$yyyymm, 5, 6)=="06", 1, 0)
z$jul <- ifelse(substr(z$yyyymm, 5, 6)=="07", 1, 0)
z$aug <- ifelse(substr(z$yyyymm, 5, 6)=="08", 1, 0)
z$sep <- ifelse(substr(z$yyyymm, 5, 6)=="09", 1, 0)
z$oct <- ifelse(substr(z$yyyymm, 5, 6)=="10", 1, 0)
z$nov <- ifelse(substr(z$yyyymm, 5, 6)=="11", 1, 0)

y <- z[['logit_perf']]
z.data <- select(z, gid, host_is_superhost, number_of_reviews, price, host_listings_count, jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov)
colnames(z.data)[1] <- "group.id"
rm(x, z, data, epsilon, percent_perf)

In [5]:
%%R
results = adjusted.predictors(x.data, z.data, x.gid, z.gid)

In [ ]:
%%R
model.formula = as.formula(y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost + number_of_reviews
                           + price + host_listings_count + jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov)
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

Call:
micromacro.lm( y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost + number_of_reviews + price + host_listings_count, ...)
    
Residuals:
       Min        1Q      Median        3Q      Max
 -18.94295 -1.190486 -0.09232311 0.9000749 16.31791

    
Coefficients:
                         Estimate Uncorrected S.E. Corrected S.E.    df
(Intercept)          0.2412389703     0.0628683557   0.0877028649 55539
BLUP.cult_dst_6     -0.0162007625     0.0457222162   0.0510914812 55539
BLUP.interaction     0.0314244107     0.0335209056   0.0347291324 55539
host_is_superhost    0.4216437442     0.0394895321   0.0410296262 55539
number_of_reviews    0.0045468309     0.0002229835   0.0002287574 55539
price               -0.0006945238     0.0001700913   0.0005355778 55539
host_listings_count  0.0009315329     0.0010552649   0.0008922749 55539
                             t     Pr(>|t|)           r
(Intercept)          2.7506396 5.949830e-03 0.011670920
BLUP.cult_dst_6     -0.3170932 7.511

In [ ]:
%%R
saveRDS(model.output, "/content/drive/My Drive/m1.rds")

# **run full data with TPU**

In [14]:
%%R
#save(y, file="/content/drive/My Drive/y.RData")
#save(results, file="/content/drive/My Drive/temp.RData")
load("/content/drive/My Drive/temp.RData")
load("/content/drive/My Drive/y.RData")

In [15]:
%%R
micromacro.lm = function(model, adjusted.predictors, y, unequal.groups=NULL) {
  data = cbind(adjusted.predictors,y)
  uncorrected.output = lm( model, data)
  model.terms = attr(terms.formula(model), "term.labels")
  model.variables = model.terms[!grepl(":",model.terms)]
  interaction.terms = model.terms[grepl(":",model.terms)]
  if (length(interaction.terms)>0) {
    interactions = matrix( NA, nrow=dim(data)[1], ncol=length(interaction.terms) )
    for ( i in 1:length(interaction.terms) ) {
      interactions[,i] = apply( data[strsplit(interaction.terms,":")[[i]]], 1, prod )
    }
    u = cbind(rep(1,dim(data)[1]),as.matrix(cbind(data[model.variables], interactions)))
  } else {u = cbind(rep(1,dim(data)[1]),as.matrix(cbind(data[model.variables])))}
  e = uncorrected.output$residuals
  p = solve(t(u) %*% u)
  h = diag(u %*% p %*% t(u))
  d = e^2/(1-h)
  v = p %*% t(u) %*% diag(d) %*% u %*% p
  se = sqrt(diag(v))
  estim = summary(uncorrected.output)$coefficients[,1:2]
  t.stats = estim[,1]/se
  p.values = pt(-abs(t.stats),rep(uncorrected.output$df.residual,length(model.terms)+1))*2 # two-sided
  effect.sizes = sqrt( t.stats**2/(t.stats**2+uncorrected.output$df.residual) )
  estim1 = cbind(estim,se,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats,p.values,effect.sizes)
  dimnames(estim1)[[2]] = c("Estimate","Uncorrected S.E.","Corrected S.E.","df","t","Pr(>|t|)","r")
  # print( estim )
  if (is.null(unequal.groups)){ # default (null) is same group size
    t.stats2 = estim[,1]/estim[,2]
    p.values2 = pt(-abs(t.stats2),rep(uncorrected.output$df.residual,length(model.terms)+1))*2
    effect.sizes2 = sqrt( t.stats2**2/(t.stats2**2+uncorrected.output$df.residual) )
    estim2 = cbind(estim,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats2,p.values2,effect.sizes2)
    dimnames(estim2)[[2]] = c("Estimate","S.E.","df","t","Pr(>|t|)","r")
    report = list("statistics" = estim2, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  } else if (isTRUE(unequal.groups)) { # true when group sizes are unequal
    report = list("statistics" = estim1, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  } else { # true when group sizes are the same
    t.stats2 = estim[,1]/estim[,2]
    p.values2 = pt(-abs(t.stats2),rep(uncorrected.output$df.residual,length(model.terms)+1))*2
    effect.sizes2 = sqrt( t.stats2**2/(t.stats2**2+uncorrected.output$df.residual) )
    estim2 = cbind(estim,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats2,p.values2,effect.sizes2)
    dimnames(estim2)[[2]] = c("Estimate","S.E.","df","t","Pr(>|t|)","r")
    report = list("statistics" = estim2, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  }
  return(report)
}

In [16]:
%%R
micromacro.summary = function(model.output) {
  cat(noquote("Call:"))
  cat("\n")
  cat(noquote(sprintf("micromacro.lm( %s %s %s, ...)", gettext(model.output$model.formula)[2], gettext(model.output$model.formula)[1], gettext(model.output$model.formula)[3])))
  cat("\n")
  cat(noquote("    "))
  cat("\n")
  # summarize the residuals
  cat(noquote("Residuals:"))
  cat("\n")
  residual.summary = cbind(min(model.output$residuals), quantile(model.output$residuals, 0.25), quantile(model.output$residuals, 0.5), quantile(model.output$residuals, 0.75), max(model.output$residuals))
  dimnames(residual.summary)[[1]] = ""
  dimnames(residual.summary)[[2]] = c("Min","1Q","Median","3Q","Max")
  print(noquote(residual.summary))
  cat("\n")
  cat(noquote("    "))
  cat("\n")
  # summarize the coefficients
  cat(noquote("Coefficients:"))
  cat("\n")
  print(noquote(model.output$statistics))
  cat("\n")
  cat(noquote("---"))
  cat("\n")
  cat(noquote(sprintf("Residual standard error: %s on %s degrees of freedom", round(sd(model.output$residuals), digits = 5), length(model.output$residuals)-dim(model.output$statistics)[1])))
  cat("\n")
  cat(noquote(sprintf("Multiple R-squared: %s, Adjusted R-squared: %s", round(model.output$rsquared, digits = 10), round(model.output$rsquared.adjusted, digits = 10))))
  cat("\n")
  cat(noquote(sprintf("F-statistic: %s on %s and %s DF, p-value: %s", round(model.output$fstatistic[1], digits = 5), model.output$fstatistic[2], model.output$fstatistic[3], round(pf(model.output$fstatistic[1], model.output$fstatistic[2], model.output$fstatistic[3], lower.tail=FALSE), digits = 5))))
}

In [17]:
%%R
model.formula = as.formula(y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost + number_of_reviews
                           + price + host_listings_count + jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov)
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

Call:
micromacro.lm( y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost + number_of_reviews + price + host_listings_count + jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov, ...)
    
Residuals:
       Min        1Q     Median        3Q      Max
 -19.69008 -1.361168 -0.3585987 0.7144378 20.37806

    
Coefficients:
                        Estimate Uncorrected S.E. Corrected S.E.    df
(Intercept)         -0.484338806     0.1087925275   0.1034392264 64941
BLUP.cult_dst_6     -0.057695511     0.0402957841   0.0435559238 64941
BLUP.interaction     0.046251607     0.0294136418   0.0301070418 64941
host_is_superhost    0.405405705     0.0344528132   0.0364932812 64941
number_of_reviews    0.005047335     0.0001997767   0.0002120960 64941
price               -0.000834269     0.0001547901   0.0006119852 64941
host_listings_count -0.002234491     0.0009462110   0.0008593094 64941
jan                 -3.054546747     0.1241431606   0.1473646189 64941
feb                 -